# Mental Health Fine Tuned Chatbot

Installing dependencies

In [ ]:
! pip install -q --no-cache-dir transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.1 MB/s eta 0:00:00


In [ ]:
! pip install -U flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 35.3 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.5.0-cp310-cp310-linux_x86_64.whl size=120823033 sha256=3335e74258645eb190597754d42c2fee391fbdeb772847f9e1de12da60450a33
  Stored in directory: /root/.cache/pip/wheels/9e/c3/22/a576eb5627fb2c30dc4679a33d67d34d922d6dbeb24a9119b2
Successfully built flash-attn


In [ ]:
! pip install -q torch peft langchain trl accelerate bitsandbytes fastapi==0.104.1 typing_extensions==4.8.0 gradio==3.41.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 

Mount the drive shared folder where the models are stored

In [ ]:
from google.colab import drive
import os
drive.mount("/content/drive")
transformer_path = "/content/drive/My Drive/transformers/"

Mounted at /content/drive


Fine tuned model import and setup

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import AutoPeftModelForCausalLM
from accelerate import Accelerator

# Quantization of the models for better performance
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16 # A100
)

# Path to the model
model = "clement-cvll/mental-health-llama2"
tokenizer = "clement-cvll/mental-health-llama2"

# HuggingFace Token
token='' # Add your token here

model = AutoModelForCausalLM.from_pretrained(
    model,
    token=token,
    quantization_config=nf4_config,
    cache_dir=transformer_path,
)

tokenizer = AutoTokenizer.from_pretrained(
    tokenizer,
    token=token,
    cache_dir=transformer_path,
    padding='max_length',
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2"
)

# Add Padding Token
tokenizer.pad_token = tokenizer.eos_token

adapter_config.json:   0%|          | 0.00/581 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

LangChain chatbot

In [ ]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline

# Pipeline setup to facilitate the use of the model and tokenizer
pipeline = pipeline(
    "text-generation",
    do_sample=True,
    temperature=0.7,
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    pad_token_id=2
)

# Integration of the pipeline with LangChain
llm = HuggingFacePipeline(pipeline=pipeline)

In [ ]:
from langchain.prompts.prompt import PromptTemplate

# Prompt engineering to improve the result quality
template = """
[INST] <<SYS>>
You are a friendly assistant, answer people questions according on the current conversation. Help people with their mental health issues. If a user expresses thoughts of self-harm, suicide, or harm to others, prioritize their safety. Encourage them to seek immediate professional help and provide emergency contact numbers when appropriate. Do not diagnose or prescribe treatments. Avoid taking sides or expressing personal opinions. Your role is to provide a safe space for users to share. Keep your anwsers short and simple, stay calm, never answer dramaticaly. You are not a professional nor certified, don't act like it.

Current conversation:
{history}
<</SYS>>

{input} [/INST]
"""

prompt = PromptTemplate(input_variables=['input', 'history'], template=template)

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# Setup of the LangChain chatbot
memory = ConversationBufferMemory(ai_prefix="Assistant")
conversation = ConversationChain(
    llm = llm,
    memory = memory,
    prompt = prompt
)

Gradio interface

In [ ]:
import gradio

# Setup the chatbot interface using Gradio
def respond(message, chat_history=""):
    global conversation
    bot_message = conversation.predict(input = message)
    chat_history = str(chat_history) + "\nUser: " + message + "\nAssistant: " + bot_message
    return bot_message

if __name__ == "__main__":
  # Run the interface
  gradio.ChatInterface(fn=respond, title="Mental Health Assistant").launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://dacf5a530dfa4bb831.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
